In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
m_crs = 'EPSG:26910'
ll_crs = 'EPSG:4326'

In [ ]:
benthic = gpd.read_file('clean_data/benthic/benthic_substrates.shp')

In [ ]:
coastline = gpd.read_file('clean_data/coastline/mainland_coast.shp') # linestring gdf
ocean = gpd.read_file('clean_data/coastline/small_coast.shp') # for plotting

counties = gpd.read_file('raw_datasets/ca_counties/ca_counties.shp').to_crs(ll_crs)
keys = ['sd', 'ora', 'la', 'ven', 'sb', 'slo', 'mon', 'scr']
county_keys = {'sd': 'San Diego',
               'ora': 'Orange',
               'la': 'Los Angeles',
               'ven': 'Ventura',
               'sb': 'Santa Barbara',
               'slo': 'San Luis Obispo',
               'mon': 'Monterey',
               'scr': 'Santa Cruz'}
county_names = [county_keys[key] for key in keys]
counties = counties.loc[[(name in county_names) for name in counties.NAME]].reset_index(drop=True)

In [ ]:
# relevant reefs are hard substrate from 0-100m depth, where recreational fishing might potentially occur
rocks = benthic.loc[(benthic.Sub == 'Hard') & (benthic.zone.isin(['0 - 30m', '30 - 100m']))].reset_index(drop=True)

In [ ]:
rocks.head()

# split reefs up and subset for within study region and within 5km of the coastline

In [ ]:
ex = rocks.filter(['zone', 'Sub', 'geometry']).explode().reset_index(drop=True)

In [ ]:
# filter for only reefs within this square
from shapely.geometry import Polygon

coords = [(-122.5,37.5), (-122.5,32.4), (-116,32.4), (-116,37.5)]
polygon = Polygon(coords)

# Create a GeoDataFrame to handle and visualize the Polygon
gdf = gpd.GeoDataFrame({'geometry': [polygon]})

# Set a Coordinate Reference System (CRS) if necessary (e.g., EPSG:4326 for lat/lon)
gdf = gdf.set_crs(ll_crs)
gdf = gdf.to_crs(ex.crs)

# filter reefs within target region
within_target = ex.geometry.intersects(gdf.geometry[0])
ex = ex.loc[within_target].reset_index(drop=True)

In [ ]:
# make a 5km buffer from the coastline and only keep reefs there
coastline = coastline.to_crs(m_crs)
buffer = coastline.geometry[0].buffer(5 * 1000)

ex = ex.to_crs(coastline.crs)
within_buffer = ex.geometry.intersects(buffer)
ex = ex.loc[ex.geometry.intersects(buffer)].reset_index(drop=True)

In [ ]:
ex = ex.to_crs(ll_crs)
coastline = coastline.to_crs(ex.crs)
len(ex)

In [ ]:
'''fig, ax = plt.subplots(figsize=(10,10))

counties.plot(ax=ax, color='red', alpha=0.5)
ocean.plot(ax=ax, color = 'lightblue')
coastline.plot(ax=ax, color='white', linewidth=1)

ex.plot(ax=ax, color = 'gray')'''

# simplify remaining geometries using the Ramer–Douglas–Peucker algorithm (shapely.simplify()

In [ ]:
g = ex.geometry[11]

In [ ]:
g.area

In [ ]:
g

In [ ]:
from shapely import simplify

In [ ]:
simplify(g, 5)

In [ ]:
# simplify within tolerance 5m
from shapely import simplify
simple = ex.copy().to_crs(m_crs)

epsilon = 5 # 5m tolerance
simple.geometry = ex.geometry.apply(lambda g: simplify(g, epsilon))

In [ ]:
simple = simple.to_crs(ll_crs)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

counties.plot(ax=ax, color='red', alpha=0.5)
ocean.plot(ax=ax, color = 'lightblue')
coastline.plot(ax=ax, color='white', linewidth=1)

simple.plot(ax=ax, color = 'gray')

In [ ]:
ex_num_vertices = [ex.geometry.apply(lambda g: len(g.exterior.coords))]

In [ ]:
simple_num_vertices = [simple.geometry.apply(lambda g: len(g.exterior.coords))]

In [ ]:
np.mean(ex_num_vertices)

In [ ]:
np.mean(simple_num_vertices)

# further simplify geometry with buffer method

In [ ]:
# buffer is at 200m
simple = simple.to_crs(m_crs)

In [ ]:
simple_buffer = simple.geometry.apply(lambda g: g.buffer(200))

In [ ]:
simple_buffer_union = simple_buffer.union_all()

In [ ]:
simple_buffer =gpd.GeoDataFrame(geometry=[simple_buffer_union], crs = simple.crs)

In [ ]:
simple_buffer_exploded = simple_buffer.explode()

In [ ]:
len(simple_buffer_exploded)

In [ ]:
simple_buffer_exploded.head()

In [ ]:
simple_buffer_exploded = simple_buffer_exploded.reset_index(drop=True).reset_index()

In [ ]:
simple_buffer_exploded.columns = ['reef_id', 'geometry']

In [ ]:
simple_buffer_exploded.head()

In [ ]:
simple_num_vertices = [simple.geometry.apply(lambda g: len(g.exterior.coords))]

# get the area of hard benthic substrate at each location

In [ ]:
centroids = ex.to_crs(m_crs)

In [ ]:
centroids['area_m2'] = centroids.geometry.area

In [ ]:
centroids.geometry = centroids.geometry.centroid

In [ ]:
centroids.head()

In [ ]:
centroids = centroids.to_crs(ll_crs)
simple_buffer_exploded = simple_buffer_exploded.to_crs(centroids.crs)

centroids['reef_id'] = [None] * len(centroids)
for i,g in enumerate(centroids.geometry):
    for j,(reef, rid) in enumerate(zip(simple_buffer_exploded.geometry,simple_buffer_exploded.reef_id)):
        if g.intersects(reef):
            centroids.loc[i, 'reef_id'] = rid

In [ ]:
centroids.reef_id.isna().sum(), centroids.reef_id.isna().mean()

# save simplified shapefiles

In [ ]:
# distinct reefs
simple_buffer_exploded.to_file('clean_data/benthic_simplified/benthic_simplified.shp')

In [ ]:
# points and reef area
centroids.to_file('clean_data/benthic_simplified/centroids.shp')